In [ ]:
import os
import uuid
from dotenv import load_dotenv
import textwrap

from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai import types

if __name__ == "__main__":
    
    # Prompt de Usuário
    user_text_input = textwrap.dedent(f"""\
        Essa é uma mensagem de teste.
        """)
    
    # Agente de Teste
    root_agent = LlmAgent(
        name='test_agent',
        model="gemini-2.5-flash",
        description="Agente de teste",
        instruction="Você é um agente de teste, responda as perguntas apenas",
    )
    
    # Carrega as Variáveis de Ambiente do .env -> Chave API
    if os.path.exists('.env'):
        load_dotenv(override=True) 
    
    user_uuid = str(uuid.uuid4())
    session_uuid = str(uuid.uuid4())
    
    # Serviço de Sessão - Não persistente
    session_service = InMemorySessionService()
    
    # Criação de Sessão
    app_session = await session_service.create_session(
        app_name="agents",
        user_id=user_uuid,
        session_id=session_uuid,
        # state= # Estado Inicial da Sessão
    )
    
    # Criação de Runner 
    runner = Runner(
        app_name=app_session.app_name,
        session_service=session_service,
        agent=root_agent
    )
    
    # Atualiza input do usuário
    user_input = []
    user_input.append(types.Part(text=user_text_input))
    user_message = types.Content(role="user", parts=user_input)
    
    # Testa o Agente
    async for event in runner.run_async(
        session_id=app_session.id,
        user_id=app_session.user_id,
        new_message=user_message
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
                break
    
    # Recarrega a sessão para atualizar os dados (state e eventos)
    app_session = await session_service.get_session(
        app_name=app_session.app_name,
        user_id=app_session.user_id,
        session_id=app_session.id
    )
    
    print(final_response_text)
    print(app_session.events)


Ok. Mensagem de teste recebida.
[Event(content=Content(
  parts=[
    Part(
      text="""Essa é uma mensagem de teste.
"""
    ),
  ],
  role='user'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-a9ad9c87-aed2-4f2c-bfaf-5b2f9521593e', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None), long_running_tool_ids=None, branch=None, id='128f8e7d-fcc3-403b-b59a-4038224c34ad', timestamp=1762720508.681859), Event(content=Content(
  parts=[
    Par